In [ ]:
# !pip install cryptography
# !pip install PyMySQL

In [25]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.debug = True
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql+pymysql://{username}:{password}@{host}:{port}/{db}?charset=utf8'.format(username="kokon",
            password="TrOlOlO123))", host="127.0.0.1", port=3306, db="prediction")
db = SQLAlchemy(app)


In [26]:
class Subject(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    results = db.relationship("Results", back_populates="subject")

class Results(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    not_succes_done = db.Column(db.Boolean, default=False)
    score = db.Column(db.Float, default=0.0)
    subject_id = db.Column(db.Integer, db.ForeignKey('subject.id'), nullable=False)
    # subject = db.relationship('Subject', backref=db.backref('results', lazy=True, cascade="all"))
    subject = db.relationship("Subject", back_populates="results", cascade="all")
    gender = db.Column(db.Integer, nullable=False, default=0)
    scores = db.relationship("Scores", back_populates="result")


class Scores(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    result_id = db.Column(db.Integer, db.ForeignKey('results.id'), nullable=False)
    # result = db.relationship('Results', backref=db.backref('scores', lazy=True, cascade="all"))
    result = db.relationship("Results", back_populates="scores", cascade="all")
    visiting = db.Column(db.Float, default=0.0)
    scores_before = db.Column(db.Float, default=0.0)
    scores_before_percent = db.Column(db.Float, default=0.0)
    scores_test = db.Column(db.Float, default=0.0)
    test_number = db.Column(db.Integer, nullable=False)

In [33]:
import tkinter.filedialog as tkFileDialog
def open_file():
    return tkFileDialog.askopenfilename( title='Выберите файл:')
import pandas as pd

data = pd.read_csv(open_file(), delimiter=";", encoding='cp1251', decimal=",") #delimiter=";", 
data = data.dropna()
data

,ФИО,Пол,Команда,Баллы,Не сдал(-а),Посещение до Контрольная работа 1,Баллы до Контрольная работа 1,Процент баллов до Контрольная работа 1,Контрольная работа 1,Посещение до Контрольная работа 2,Баллы до Контрольная работа 2,Процент баллов до Контрольная работа 2,Контрольная работа 2,Посещение до Контрольная работа 3,Баллы до Контрольная работа 3,Процент баллов до Контрольная работа 3,Контрольная работа 3
0,Абатуров Георгий Сергеевич,1.0,ПиОА Л-08,76.0,0,0.93,16.0,0.52,12.0,1.0,6.0,0.26,12.0,0.43,10.0,0.43,12.0
1,Абдимиталипов Мустафа,1.0,ПиОА Л-04,24.07,0,1.0,3.5,0.11,7.38,1.0,1.2,0.05,4.49,1.0,0.0,0.0,7.5
2,Абдимуминов Абдужалол Абдихаким угли,1.0,ПиОА Л-05,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Абдразакова Екатерина Наилевна,0.0,ПиОА Л-04,87.67,0,1.0,15.0,0.48,11.7,1.0,6.0,0.26,11.8,1.0,8.0,0.35,15.5
4,Абдрашитова Диана Дамировна,0.0,ПиОА Л-07,63.56,0,0.86,16.0,0.52,4.48,0.5,6.0,0.26,10.08,1.0,8.0,0.35,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,Ярмоц Надежда Андреевна,0.0,ПиОА Л-08,65.0,0,1.0,9.0,0.29,7.0,0.5,7.0,0.3,9.0,0.86,11.5,0.5,10.0
957,Ярославцев Дмитрий Сергеевич,1.0,ПиОА Л-04,33.69,1,0.86,5.0,0.16,8.02,0.5,4.0,0.17,8.77,0.43,0.0,0.0,1.5
958,Яруллин Тимофей Русланович,1.0,ПиОА Л-08,62.51,0,0.79,16.0,0.52,7.0,0.5,6.0,0.26,7.01,1.0,10.0,0.43,8.5
959,Ятло Николай Олегович,1.0,ПиОА Л-04,19.43,1,0.79,4.0,0.13,7.13,1.0,1.0,0.04,0.0,0.43,2.0,0.09,0.0


In [4]:
with app.app_context() as ctx:
    ctx.push()
    db.create_all()

In [34]:
subject_temp = Subject(name="Программирование и основы алгоритмизации 1")
with app.app_context():
    db.session.add_all([subject_temp])
    db.session.commit()

In [35]:
exams = [name  for name in data.columns.to_list() if name.startswith("Контрольная")]
with app.app_context():
    for index, d in data.iterrows():
        result = Results(not_succes_done= d["Не сдал(-а)"]==1.0, score=d[f"Баллы"],subject=subject_temp, gender = d[f"Пол"])
        db.session.add_all([result])
        db.session.commit()
        for ex in exams:
            score = Scores(result=result, visiting=d[f"Посещение до {ex}"], scores_test = d[ex], test_number=exams.index(ex)+1)
            if (f"Баллы до {ex}" in data.columns):
                score.scores_before = d[f"Баллы до {ex}"]
                score.scores_before_percent = d[f"Процент баллов до {ex}"]
            db.session.add_all([score])
            db.session.commit()